In [55]:
import pandas as pd
import numpy as np
import sklearn

#  **数据预处理**

In [56]:
data = pd.read_csv('ml2020spring-hw1/train.csv',encoding="big5")

In [57]:
#去掉前三列不需要的数据，并且将RAINFALL对应的NR改为0
data = data.iloc[:, 3:]
data[data=='NR'] = 0

In [58]:
raw_data = data.to_numpy()
raw_data.shape

(4320, 24)

# **特征工程**

# （1）按月份来处理数据
* 针对每20天来说，信息维度[18, 480] (18个feature，20*24=480)
* 将原始的数据按照每个月来划分，重组成12个 [18,480]

In [59]:
month_data = {} #建立字典{key:month value:data}
for month in range(12):
    sample = np.empty([18,20 * 24]) #创建空的数组用于临时保存每个月的数据
    for day in range(20):
        sample[:,day * 24 : (day+1) * 24] = raw_data[18 * (20 * month + day):18 * (20 * month + day + 1),:]
    month_data[month] = sample

In [60]:
month_data

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
        [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
        [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
        ...,
        [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
        [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
        [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]),
 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
        [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
        [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
        ...,
        [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
        [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
        [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]),
 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
        [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
        [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
        ...,
        [103.  , 128.  , 115.  , ...,  

In [61]:
# 具体数据(以第一个月为例)
print(month_data[0])
month_data[0].shape

[[14.   14.   14.   ... 14.   13.   13.  ]
 [ 1.8   1.8   1.8  ...  1.8   1.8   1.8 ]
 [ 0.51  0.41  0.39 ...  0.34  0.41  0.43]
 ...
 [35.   79.    2.4  ... 48.   63.   53.  ]
 [ 1.4   1.8   1.   ...  1.1   1.9   1.9 ]
 [ 0.5   0.9   0.6  ...  1.2   1.2   1.3 ]]


(18, 480)

# （2）扩充数据集，获取更好的训练效果
* 根据实验要求，需要用连续9个时间点的数据预测第10个时间点的PM2.5。 而每个月采取的是前20天连续的数据，可以扩展成480小时的连续数据；
* 具体做法，每个月的第一天的0-8时的数据作为训练数据，9时的数据作标签y；9-17的数据作一个data，18时的数据作标签y.....以此推，每个月480小时，有480-9= 471个data，故此时总数据471 \* 12 个；而每个data是18*9

In [62]:
# 创建两个空数组，分别用于保存数据与标签
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
# 将数据填入x,y
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day==19 and hour > 14:
                continue
                # 转而以一维序列代替，一维序列的顺序本身可以隐含其时序信息
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) 
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x.shape)
y.shape

(5652, 162)


(5652, 1)

In [63]:
mean_x = np.mean(x,axis=0) # 求均值，axis=0表示沿着列计算
std_x = np.std(x,axis=0) #求标准差
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j] #将属性值进行归一化，避免数据的某些特征值形成主导作用

In [110]:
mean_x.shape

(162,)

# 损失函数
* 采用预测值与标签y之间的平均欧时距离来衡量预测的准确程度 jupyter
* num = 471*12， 乘 1/2 是为了在后续求梯度过程中保证梯度系数为1，方便计算

# 学习率更新

为了在不影响模型效果的前提下提高学习速度，可以对学习率进行实时更新：即让学习率的值在学习初期较大，之后逐渐减小。这里采用比较经典的adagrad算法来更新学习率。

In [64]:
dim = x.shape[1] + 1 #此处+1是为了保留其中的一个常数项？
w = np.zeros(shape = (dim, 1 )) #empty创建的数组，数组中的数取决于数组在内存中的位置处的值，为0纯属巧合？
x = np.concatenate((np.ones((x.shape[0], 1 )), x) , axis = 1).astype(float) 

In [65]:

#初始化学习率(163个参数，163个200)和adagrad
learning_rate = np.array([[200]] * dim)
adagrad_sum = np.zeros(shape = (dim, 1 ))
 
#没有隐藏层的网络
for T in range(10001):
    if(T % 500 == 0 ):
        print("T=",T)
        print("Loss:",np.sum((x.dot(w) - y)**2)/ x.shape[0] /2) #最小二乘损失
        print((x.dot(w) - y)**2)
    gradient = 2 * np.transpose(x).dot(x.dot(w)-y) #损失的导数x*(yh-h)
    adagrad_sum += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad_sum) + 0.0005)

np.save('weight.npy',w)

T= 0
Loss: 366.4253361641897
[[ 900.]
 [1681.]
 [1936.]
 ...
 [ 289.]
 [ 576.]
 [ 841.]]
T= 500
Loss: 117.09639332560212
[[ 4.11916821]
 [77.85067962]
 [19.36061088]
 ...
 [65.73125473]
 [ 8.48470051]
 [41.46193399]]
T= 1000
Loss: 49.57427263343191
[[11.54893258]
 [ 6.80609203]
 [ 2.22845136]
 ...
 [47.65526993]
 [ 0.78188001]
 [57.53683525]]
T= 1500
Loss: 31.041919746294443
[[36.67506759]
 [ 0.19789921]
 [ 0.61279255]
 ...
 [27.49953885]
 [ 0.20822184]
 [50.43354985]]
T= 2000
Loss: 23.539993089166042
[[48.87757904]
 [ 0.39734139]
 [ 0.47535117]
 ...
 [18.98454253]
 [ 1.34155635]
 [42.62481417]]
T= 2500
Loss: 20.091482565687592
[[53.56499952]
 [ 1.64351092]
 [ 0.83495479]
 ...
 [15.03708605]
 [ 2.42090458]
 [36.18012439]]
T= 3000
Loss: 18.3651940614608
[[55.57335778]
 [ 2.98203704]
 [ 1.5816199 ]
 ...
 [13.00961386]
 [ 3.28490111]
 [31.1598156 ]]
T= 3500
Loss: 17.44596507038221
[[56.87752076]
 [ 4.20374056]
 [ 2.61128111]
 ...
 [11.89724046]
 [ 3.99602983]
 [27.3273404 ]]
T= 4000
Loss:

In [66]:
# 查看模型各权重
w

array([[ 2.13740269e+01],
       [ 2.07294854e-01],
       [-6.41640494e-01],
       [ 1.05456366e+00],
       [-1.36948084e+00],
       [-7.74411109e-01],
       [ 1.79598073e+00],
       [-7.31754648e-01],
       [-1.68526128e+00],
       [ 2.05123116e+00],
       [-3.29936822e-01],
       [ 1.38064158e-01],
       [ 7.06223655e-02],
       [ 1.06410861e-01],
       [-3.72837598e-02],
       [-2.03733091e-02],
       [-1.78629730e-01],
       [ 1.62377879e-01],
       [ 5.22518275e-01],
       [ 4.91299479e-02],
       [-2.42588101e-02],
       [ 6.62554559e-02],
       [-1.58783675e-01],
       [ 1.50982156e-01],
       [-1.73019236e-02],
       [-1.62357561e-01],
       [ 7.62278855e-02],
       [ 4.03878860e-01],
       [-3.11212237e-01],
       [ 3.72754893e-01],
       [-2.85451803e-01],
       [ 3.24489166e-01],
       [ 2.91503606e-01],
       [-5.19142672e-01],
       [ 9.36602067e-02],
       [ 1.89134220e-01],
       [ 8.35214402e-02],
       [-3.64171572e-02],
       [-1.3

# 使用模型预测test.csv

In [108]:
data = pd.read_csv('ml2020spring-hw1/test.csv',header=None,encoding="big5")
test_data = data.iloc[:,2:]
test_data = test_data.copy()
test_data[test_data=="NR"] = 0
test_data = np.array(test_data)
test_data.shape

(4320, 9)

In [111]:
# 载入模型
w = np.load('weight.npy')
x_test = np.empty(shape = (240,18*9),dtype=float)

for i in range(240):
    x_test[i,:] = test_data[18 * i : 18 * (i+1),:].reshape(1,-1)

mean_test = np.mean(x_test,axis = 0)
std_test = np.std(x_test,axis = 0)

# 同样的，测试集也需要归一化
for i in range(x_test.shape[0]):
    for j in range(x_test.shape[1]):
        if not std_x[j] == 0:
            x_test[i][j] = (x_test[i][j] - mean_test[j])/std_test[j]
            
x_test = np.concatenate((np.ones(shape = (x_test.shape[0],1)),x_test),axis = 1).astype(float)

In [69]:
print(w.shape)
x_test.shape

(163, 1)


(240, 163)

# 利用模型求出测试集结果

In [70]:
ans_y = np.dot(x_test,w)

# 将结果写入CSV文件

In [71]:
import csv
with open("result.csv",mode='w',newline='') as file:
    csv_writer = csv.writer(file)
    header = ['id','value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id'+'_'+str(i),ans_y[i][0]]
        csv_writer.writerow(row)

# 优化?BaseLine

In [112]:
from sklearn import linear_model

In [113]:
print(y.shape)
x.shape

(5652, 1)


(5652, 163)

In [114]:
x_test.shape

(240, 163)

In [115]:
model = linear_model.Ridge(alpha = 0.0001)
model.fit(x,y)

Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [116]:
rr_y_predict = model.predict(x_test)
rr_y_predict.shape

(240, 1)

In [117]:
import csv
with open("result_ridge.csv",mode='w',newline='') as file:
    csv_writer = csv.writer(file)
    header = ['id','value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id'+'_'+str(i),rr_y_predict[i][0]]
        csv_writer.writerow(row)

# 第二次优化？

In [78]:
print(x.shape)
print(y.shape)
print(x_test.shape)

(5652, 163)
(5652, 1)
(240, 163)


In [79]:
model = linear_model.LinearRegression()
model.fit(x,y)
lr_result = model.predict(x_test)
with open("result_nomal.csv",mode='w',newline='') as file:
    csv_writer = csv.writer(file)
    header = ['id','value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id'+'_'+str(i),lr_result[i][0]]
        csv_writer.writerow(row)

** 不想再往下写标题了，，**

In [80]:
import csv
# 结果数据写入函数
def result_csv(file_name,predict_result):
    with open(file_name,mode='w',newline='') as file:
        csv_writer = csv.writer(file)
        header = ['id','value']
        csv_writer.writerow(header)
        for i in range(240):
            row = ['id'+'_'+str(i),predict_result[i]]
            csv_writer.writerow(row)

In [86]:
print(x.shape)
print(y.shape)
print(x_test.shape)

(5652, 163)
(5652, 1)
(240, 163)


* **切分数据集** 

In [87]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,random_state=0,test_size=0.10)


In [100]:
from sklearn.metrics import r2_score
model_lasso = linear_model.Lasso(alpha = 0.0002)
model_lasso.fit(x,y)
# result_lasso = model_lasso.predict(x_test)
# result_csv("result_lasso.csv",result_lasso)
result_lasso = model_lasso.predict(X_test)
score = r2_score(y_test,result_lasso)
score

0.8545764417286673

# SVM

In [81]:
from sklearn.svm import SVR

In [84]:
model_rbf = SVR(kernel="rbf",C=100, gamma=0.1,epsilon=.01) #高斯核
model_lin = SVR(kernel="linear",C=100,gamma="auto") #线性核
model_poly =SVR(kernel="poly",C=100,gamma='auto',degree=3,epsilon=.01,coef0=1) #径向基核函数
model_rbf.fit(x,y)
model_lin.fit(x,y)
model_poly.fit(x,y)

SVR(C=100, cache_size=200, coef0=1, degree=3, epsilon=0.01, gamma='auto',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [85]:
rbf_result = model_rbf.predict(x_test)
lin_result = model_lin.predict(x_test)
poly_result = model_poly.predict(x_test)
result_csv("rbf_result.csv",rbf_result)
result_csv("lin_result.csv",lin_result)
result_csv("poly_result.csv",poly_result)

In [107]:
model_poly_2 =SVR(kernel="poly",C=1,gamma='scale',degree=4,epsilon=.0001,coef0=1) #径向基核函数
model_poly_2.fit(X_train,y_train)
result_poly_2= model_poly_2.predict(X_test)
score = r2_score(y_test,result_poly_2)
score

0.8150108660388333

# 第三次优化？

** 行吧，，还得写，，，**
** 这次不成功就用tf了 **